In [8]:
import pandas as pd
from pathlib import Path
from pprint import pprint
from datetime import datetime, timedelta # to convert timestamp/ID to human readable format

In [2]:

raw_data = Path('rawdata')
tables = {}
for i, file in enumerate(raw_data.iterdir()):
    name = file.name.split('.')[0]
    tables[f"{name}"] = pd.read_csv(f"rawdata/{file.name}", header = None, names=('ID', name))


In [3]:
columns = [i for i in tables.keys()] 
columns.sort()
pprint(columns)

['fuelDensity',
 'fuelTemp',
 'fuelVolumeFlowRate',
 'inclinometer-raw',
 'latitude',
 'level1median',
 'level2median',
 'longitude',
 'longitudinalWaterSpeed',
 'portPitch',
 'portRudder',
 'speedKmh',
 'speedKnots',
 'starboardPitch',
 'starboardRudder',
 'trackDegreeMagnetic',
 'trackDegreeTrue',
 'trueHeading',
 'windAngle',
 'windSpeed']


### Date transformation functions

In [4]:
def ticks_to_iso(ticks):
    """Convert .NET DateTime.Ticks to an ISO 8601 formatted string."""
    ticks_epoch = datetime(1, 1, 1)  # .NET starts from year 0001-01-01
    date_time = ticks_epoch + timedelta(microseconds=ticks / 10)  # Convert ticks to microseconds
    return date_time.isoformat()

def ticks_to_ymd(ticks):
    """Convert .NET DateTime.Ticks to an ISO 8601 formatted string."""
    ticks_epoch = datetime(1, 1, 1)  # .NET starts from year 0001-01-01
    date_time = ticks_epoch + timedelta(microseconds=ticks / 10)  # Convert ticks to microseconds
    return date_time.strftime("%Y-%m-%d")

# Adds date and ISO date to table in tables dictionary
def add_dates(table_name): 
    tables[table_name]["date"] = tables[table_name].ID.apply(ticks_to_ymd)
    tables[table_name]["iso_date"] = tables[table_name].ID.apply(ticks_to_iso)

### Aggregting date for mass flow of fuel

In [5]:
df_cons = pd.DataFrame(tables["fuelDensity"].ID.tolist(), columns = ('ID',))
df_cons = df_cons.merge(tables["fuelDensity"], on='ID',how='inner')
df_cons = df_cons.merge(tables["fuelVolumeFlowRate"], on='ID',how='inner')
df_cons

,ID,fuelDensity,fuelVolumeFlowRate
0,634019142119225390,0.947109,0.000000
1,634019142129597610,0.947110,0.000000
2,634019142139821660,0.947114,0.000000
3,634019142150036040,0.947104,0.000000
4,634019212935606850,0.938305,0.283816
...,...,...,...
1627319,634067079121428610,0.930151,0.203576
1627320,634067079131642650,0.930103,0.205217
1627321,634067079141866640,0.930075,0.212853
1627322,634067079152080710,0.930082,0.224660


In [9]:
df_cons["date"] = df_cons.ID.apply(ticks_to_ymd)
df_cons["iso_date"] = df_cons.ID.apply(ticks_to_iso)

In [10]:
df_cons

,ID,fuelDensity,fuelVolumeFlowRate,date,iso_date
0,634019142119225390,0.947109,0.000000,2010-02-16,2010-02-16T10:50:11.922536
1,634019142129597610,0.947110,0.000000,2010-02-16,2010-02-16T10:50:12.959760
2,634019142139821660,0.947114,0.000000,2010-02-16,2010-02-16T10:50:13.982168
3,634019142150036040,0.947104,0.000000,2010-02-16,2010-02-16T10:50:15.003600
4,634019212935606850,0.938305,0.283816,2010-02-16,2010-02-16T12:48:13.560688
...,...,...,...,...,...
1627319,634067079121428610,0.930151,0.203576,2010-04-12,2010-04-12T22:25:12.142864
1627320,634067079131642650,0.930103,0.205217,2010-04-12,2010-04-12T22:25:13.164264
1627321,634067079141866640,0.930075,0.212853,2010-04-12,2010-04-12T22:25:14.186664
1627322,634067079152080710,0.930082,0.224660,2010-04-12,2010-04-12T22:25:15.208072


In [11]:
display(df_cons[(df_cons["fuelVolumeFlowRate"] > 0) & (df_cons["date"] == '2010-02-16')])
display(df_cons[(df_cons["fuelVolumeFlowRate"] > 0) &  (df_cons["date"] == '2010-02-17')])

,ID,fuelDensity,fuelVolumeFlowRate,date,iso_date
4,634019212935606850,0.938305,0.283816,2010-02-16,2010-02-16T12:48:13.560688
5,634019212945820680,0.938173,0.279202,2010-02-16,2010-02-16T12:48:14.582064
6,634019212956034910,0.938036,0.282821,2010-02-16,2010-02-16T12:48:15.603488
7,634019212966268860,0.937983,0.280900,2010-02-16,2010-02-16T12:48:16.626888
8,634019212976492840,0.937767,0.279850,2010-02-16,2010-02-16T12:48:17.649280
...,...,...,...,...,...
8634,634019301152989210,0.926645,0.185059,2010-02-16,2010-02-16T15:15:15.298920
8635,634019301163232170,0.926666,0.185255,2010-02-16,2010-02-16T15:15:16.323216
8636,634019301173466570,0.926686,0.187334,2010-02-16,2010-02-16T15:15:17.346656
8637,634019301183680670,0.926721,0.185752,2010-02-16,2010-02-16T15:15:18.368064


,ID,fuelDensity,fuelVolumeFlowRate,date,iso_date
8639,634019858471552070,0.935706,0.394521,2010-02-17,2010-02-17T06:44:07.155208
8640,634019858481776080,0.935774,0.347631,2010-02-17,2010-02-17T06:44:08.177608
8641,634019858492000120,0.935693,0.327961,2010-02-17,2010-02-17T06:44:09.200016
8642,634019858502214030,0.935598,0.317626,2010-02-17,2010-02-17T06:44:10.221400
8643,634019858512448020,0.935773,0.311582,2010-02-17,2010-02-17T06:44:11.244800
...,...,...,...,...,...
36613,634020383895512390,0.925759,0.221838,2010-02-17,2010-02-17T21:19:49.551240
36614,634020383905736690,0.925743,0.221843,2010-02-17,2010-02-17T21:19:50.573672
36615,634020383915950290,0.925741,0.215218,2010-02-17,2010-02-17T21:19:51.595032
36616,634020383926174300,0.925739,0.216107,2010-02-17,2010-02-17T21:19:52.617432


#### Adding data about motion
Since fuel density is implied by fuel temperature. These variables are correlated and adding tempereture would not add any information. I will rather add location as it can be more related to the weather conditions, that can have influence on fuel mass flow. Since joining on time stamp is not an option in this case, I will transform it to human readable format and 

In [ ]:
add_dates("longitude")
add_dates("latitude")

In [ ]:
df_longitude = tables["longitude"]
df_longitude["iso_date_min"] = df_longitude.iso_date.apply(lambda x: x[:16])
df_longitude

In [ ]:
print("First fuel consumption timestemp 2010-02-16T12:48:13.560688")
print("Longitude")
display(df_longitude[df_longitude.iso_date_min == '2010-02-16T12:48'])
print("Latitude")
display(tables["latitude"])

In [ ]:
# I cannot joinon date ofc
'''
df_gps = pd.DataFrame(tables["latitude"].date.tolist(), columns = ('date',))
df_gps = df_gps.merge(tables["latitude"].drop("ID", axis=1), on='date',how='inner')
df_gps = df_gps.merge(tables["longitude"].drop("ID", axis=1), on='date',how='inner')
df_gps
'''

In [ ]:
display(tables["latitude"].drop("ID", axis=1))

In [ ]:
tables["trueHeading"].ID.apply

## Count records per day

In [20]:
df_rec_counts = None
for name, df in tables.items():
    print(f'Processing {name}')
    if name not in df.keys():
        add_dates(name)
    print('Counting...')
    df = df[["date", name]].groupby("date").count()
    
    if df_rec_counts is None:
        df_rec_counts = df
    else:
        df_rec_counts = pd.merge(df_rec_counts, df, on="date")

display(df_rec_counts)
df_rec_counts.to_csv('records_per_day.csv', header=True)

Processing starboardRudder
Counting...
Processing portPitch
Counting...
Processing level2median
Counting...
Processing fuelDensity
Counting...
Processing latitude
Counting...
Processing speedKnots
Counting...
Processing longitudinalWaterSpeed
Counting...
Processing trackDegreeTrue
Counting...
Processing windSpeed
Counting...
Processing trueHeading
Counting...
Processing starboardPitch
Counting...
Processing portRudder
Counting...
Processing longitude
Counting...
Processing fuelVolumeFlowRate
Counting...
Processing level1median
Counting...
Processing trackDegreeMagnetic
Counting...
Processing inclinometer-raw
Counting...
Processing windAngle
Counting...
Processing speedKmh
Counting...
Processing fuelTemp
Counting...


,starboardRudder,portPitch,level2median,fuelDensity,latitude,speedKnots,longitudinalWaterSpeed,trackDegreeTrue,windSpeed,trueHeading,starboardPitch,portRudder,longitude,fuelVolumeFlowRate,level1median,trackDegreeMagnetic,inclinometer-raw,windAngle,speedKmh,fuelTemp
date,,,,,,,,,,,,,,,,,,,,
2010-02-16,18557,18557,37535,8639,28748,18972,5894,18969,9283,15468,18557,18557,28748,8639,37902,18969,38242,9283,18972,8639
2010-02-17,28579,28579,60166,27979,44339,29247,9083,29240,14299,23833,28579,28579,44337,27979,63204,29240,58934,14299,29247,27979
2010-02-18,25999,25999,55414,25270,40377,26627,8255,26620,13009,21673,25999,25999,40375,25270,60086,26619,53587,13009,26627,25270
2010-02-19,35029,35029,70955,34294,54387,35885,11130,35873,17530,29216,35029,35029,54388,34294,73426,35874,72241,17530,35886,34294
2010-02-20,35461,35461,72709,34724,55045,36347,11271,36341,17753,29584,35461,35461,55045,34724,76399,36341,73143,17753,36348,34724
2010-02-21,29608,29608,48878,28988,45936,30302,9407,30296,14818,24696,29608,29608,45936,28988,45516,30296,61059,14818,30302,28988
2010-02-22,39748,39748,81014,34509,61703,40666,12629,40656,19892,33148,39748,39748,61704,34509,88646,40657,81964,19892,40665,34509
2010-02-23,26567,26567,58848,19360,41184,27197,8440,27190,13294,22152,26567,26567,41184,19360,63085,27190,54774,13294,27197,19360
2010-02-24,29573,29573,61634,28953,45826,30220,9395,30211,14797,24662,29573,29573,45827,28953,66193,30211,60985,14797,30220,28953


In [25]:
print(df_rec_counts.shape)
display(df_rec_counts.info())
display(df_rec_counts.describe())

(54, 20)
<class 'pandas.core.frame.DataFrame'>
Index: 54 entries, 2010-02-16 to 2010-04-12
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   starboardRudder         54 non-null     int64
 1   portPitch               54 non-null     int64
 2   level2median            54 non-null     int64
 3   fuelDensity             54 non-null     int64
 4   latitude                54 non-null     int64
 5   speedKnots              54 non-null     int64
 6   longitudinalWaterSpeed  54 non-null     int64
 7   trackDegreeTrue         54 non-null     int64
 8   windSpeed               54 non-null     int64
 9   trueHeading             54 non-null     int64
 10  starboardPitch          54 non-null     int64
 11  portRudder              54 non-null     int64
 12  longitude               54 non-null     int64
 13  fuelVolumeFlowRate      54 non-null     int64
 14  level1median            54 non-null     int64
 15  trac

None

,starboardRudder,portPitch,level2median,fuelDensity,latitude,speedKnots,longitudinalWaterSpeed,trackDegreeTrue,windSpeed,trueHeading,starboardPitch,portRudder,longitude,fuelVolumeFlowRate,level1median,trackDegreeMagnetic,inclinometer-raw,windAngle,speedKmh,fuelTemp
count,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000
mean,31173.685185,31173.685185,69348.037037,30135.629630,48361.314815,31911.055556,9904.425926,31903.314815,15599.814815,25998.796296,31173.685185,31173.685185,48361.000000,30135.629630,71941.259259,31903.222222,64289.500000,15599.814815,31911.037037,30135.629630
std,5703.044397,5703.044397,12945.812593,6176.769594,8849.599353,5838.206158,1812.329051,5836.465821,2854.238015,4756.987057,5703.044397,5703.044397,8849.681286,6176.769594,13461.877245,5836.658312,11765.468732,2854.238015,5838.065430,6176.769594
min,18557.000000,18557.000000,37535.000000,8639.000000,28748.000000,18972.000000,5894.000000,18969.000000,9283.000000,15468.000000,18557.000000,18557.000000,28748.000000,8639.000000,37902.000000,18969.000000,38242.000000,9283.000000,18972.000000,8639.000000
25%,26582.750000,26582.750000,59639.750000,26025.000000,41229.500000,27208.000000,8444.250000,27201.000000,13302.250000,22170.250000,26582.750000,26582.750000,41228.500000,26025.000000,62842.750000,27201.000000,54819.250000,13302.250000,27208.000000,26025.000000
50%,29485.000000,29485.000000,66197.000000,28867.500000,45698.500000,30151.500000,9368.000000,30142.000000,14754.500000,24591.500000,29485.000000,29485.000000,45699.000000,28867.500000,67725.500000,30142.000000,60804.000000,14754.500000,30151.500000,28867.500000
75%,34650.500000,34650.500000,78407.500000,33924.000000,53774.500000,35486.250000,11010.000000,35475.750000,17340.250000,28900.250000,34650.500000,34650.500000,53775.000000,33924.000000,80354.000000,35476.750000,71463.250000,17340.250000,35487.000000,33924.000000
max,44808.000000,44808.000000,100880.000000,43872.000000,69505.000000,45806.000000,14238.000000,45791.000000,22423.000000,37371.000000,44808.000000,44808.000000,69505.000000,43872.000000,104883.000000,45791.000000,92442.000000,22423.000000,45806.000000,43872.000000
